In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as spstats
from statsmodels.base.model import GenericLikelihoodModel
%matplotlib inline

See the paper "On fluctuation analysis: a new, simple and efficient method for computing the expected number of mutants" by Sarkar, Ma, and Sandri to understand how to calculate the probability mass function for a Luria Delbruck distribution in the limit of rare mutations in a large population. In particular, we also assume that the final population size is much much greater (approximately 1000 fold or more times) than the initial population size since this approximation simplifies things slightly and our initial population sizes in experiments are a 100,000 to 1,000,000 times less than our final population size.

In [2]:
class luriaDelbruck(spstats.rv_discrete):
    
    def __init__(self, a=0, b=np.inf, name=None, badvalue=None, moment_tol=1e-08, values=None, inc=1,
                 longname=None, shapes=None, extradoc=None, seed=None):
        spstats.rv_discrete.__init__(self, a, b, name, badvalue, moment_tol, values, inc, longname,
                                    shapes, extradoc, seed)
        self.muNk_dict = {}


    # note to self: this implementation needs to be able to take numpy arrays in for k, mu, and N in order for
    # scipy.stats machinery to be able to automatically implement the remaining methods for the probability
    # distribution correctly.In particular we need the logpdf function to work. Then we can use
    # statsmodels to automagically get maximum likelihood estimation and bootstrapping working for free.
    def _pmf(self, k, mu, N):
        soln = np.zeros_like(k,dtype='float')
        mu = np.array(mu).flatten()
        N = np.array(N,dtype='int64').flatten()
        for index, num in enumerate(k):
            if mu.size > 1:
                mu_i = mu[index]
            else:
                mu_i = mu
            if N.size > 1:
                N_i = N[index]
            else:
                N_i = N
            try:
                key = (mu_i*N_i)[0]
            except IndexError:
                key = mu_i*N_i
            if key in self.muNk_dict:
                if self.muNk_dict[key].size <= num:
                    temp = self.muNk_dict[key]
                    self.muNk_dict[key]=-1*np.ones(int(num)+1)
                    self.muNk_dict[key][0:temp.size]=temp
            else:
                self.muNk_dict[key]=-1*np.ones(int(num)+1)
            soln[index]= self._xtraprivatepmf(num, mu_i, N_i)
        return soln
    
    #this private implementation of the probability mass function only accepts single numbers not arrays.
    #We're stuck with this because we can only compute the pmf recursively.
    def _xtraprivatepmf(self, k, mu, N):
        try:
            key = (mu*N)[0]
        except IndexError:
            key = mu*N
        k=int(k)
        if self.muNk_dict[key][k] == -1:
            if k == 0:
                self.muNk_dict[key][k] = np.exp(-mu*N)
            else:
                prev_results = []
                for i in range(0,k):
                    prev_results.append(self._xtraprivatepmf(i, mu, N)/(k-i+1))
                self.muNk_dict[key][k] = mu*N/k * np.sum(np.array(prev_results))
        return self.muNk_dict[key][k]

In [3]:
sampler = luriaDelbruck()
mu_s = 10**-8
N_little = 10**8
Ratio = 10
N_big = Ratio*N_little
sample_size = (10,10)
test_sample = sampler.rvs(mu_s,N_little,size=sample_size)

In [4]:
%timeit test_sample_big = sampler.rvs(mu_s,N_big,size=sample_size)

The slowest run took 15.49 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 189 ms per loop


In [9]:
def time_pmf():
    sampler = luriaDelbruck()
    ans = sampler.pmf(np.arange(5000),10**-7,10**8)
    print(ans[4999])

In [10]:
%timeit time_pmf()

4.12472741338e-07
4.12472741338e-07
4.12472741338e-07
4.12472741338e-07
1 loop, best of 3: 29.3 s per loop


In [ ]:
test_sample_big_plated = np.random.poisson(test_sample_big/Ratio)
print(test_sample)
print(test_sample_big_plated)

In [ ]:
print(test_sample_big)

In [ ]:
medians_little = np.median(test_sample,axis=0)
medians_big_plated = np.median(test_sample_big_plated,axis=0)
print(medians_little)
print(medians_big_plated)
print('the fold difference between highest and lowest medians in current procedure is',
      np.max(medians_little)/np.min(medians_little))
print('the fold difference between highest and lowest medians in new procedure is',
      np.max(medians_big_plated)/np.min(medians_big_plated))

In [ ]:
class SarkarMaSandri(GenericLikelihoodModel):

    
    def __init__(self, endog, exog, **kwds):
        super(SarkarMaSandri, self).__init__(endog, exog, **kwds)
        self.luria_delbruck = luriaDelbruck()


    def nloglikeobs(self, params):
        mu = params[0]
        return -np.sum(np.log(self.luria_delbruck.pmf(self.endog, mu, self.exog)))


    def fit(self, start_params=None, maxiter=10000, maxfun=50, **kwds):
        if start_params == None:
            mu_start = np.sum(endog)/np.sum(exog)
            start_params = mu_start
        return super(SarkarMaSandri, self).fit(start_params=start_params,
                     maxiter=maxiter, maxfun=maxfun, **kwds)

In [ ]:
endog = test_sample_big[3] #rifampicin plates
exog = np.ones(sample_size[0])*N_big #LB plates
estimator_test = SarkarMaSandri(endog, exog)

In [ ]:
results = estimator_test.fit()
print('the estimated mutation rate is', results.params[0])

In [ ]:
estimator_test.hessian(np.array([6.5*10**-9]))

In [ ]:
estimator_test.luria_delbruck._pmf(estimator_test.endog, np.ones_like(estimator_test.endog)*1.2*10**-8,
                                   estimator_test.exog)

In [ ]:
import statsmodels.tools.numdiff as stnud

In [ ]:
stnud.approx_hess(np.array([1.2*10**-8]), estimator_test.loglike)

In [ ]:
mu = 1.89*10**-8
probs = ldcheck.pmf(endog, mu, exog)
print(-np.sum(np.log(probs)))

In [ ]:
np.sum(endog)/np.sum(exog)

In [ ]:
rif_plates_exp_1 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,2,1,8,4,3,2,9,6,30,40,1]

In [ ]:
mu = .062 * 10**(-7)
N = (51+22+23+45+59)/5*2.5*10**6
likelihoods = ldcheck.pmf(rif_plates_exp_1, mu, N)
loglikelihood = np.log(likelihoods)
print(np.sum(loglikelihood))

In [ ]:
mu = .034 * 10**(-7)
N = (51+22+23+45+59)/5*2.5*10**6
likelihoods = ldcheck.pmf(rif_plates_exp_1, mu, N)
loglikelihood = np.log(likelihoods)
print(np.sum(loglikelihood))

In [ ]:
mu = .096 * 10**(-7)
N = (51+22+23+45+59)/5*2.5*10**6
likelihoods = ldcheck.pmf(rif_plates_exp_1, mu, N)
loglikelihood = np.log(likelihoods)
print(np.sum(loglikelihood))

In [ ]:
mu = .034 * 10**(-8)
N = (51+22+23+45+59)/5*2.5*10**6
likelihoods = ldcheck.pmf(rif_plates_exp_1, mu, N)
loglikelihood = np.log(likelihoods)
print(np.sum(loglikelihood))

In [ ]:
mu = .096 * 10**(-6)
N = (51+22+23+45+59)/5*2.5*10**6
likelihoods = ldcheck.pmf(rif_plates_exp_1, mu, N)
loglikelihood = np.log(likelihoods)
print(np.sum(loglikelihood))

In [ ]:
rif_plates_exp_2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,14,100,1,1,2,2,2,1,13,6,9,2,11,62,22,8,3]

In [ ]:
mu = .079 * 10**(-7)
N = (61+17+78+37)/4*2.5*10**6
likelihoods = ldcheck.pmf(rif_plates_exp_2, mu, N)
loglikelihood = np.log(likelihoods)
print(np.sum(loglikelihood))

In [ ]:
mu = .048 * 10**(-7)
N = (61+17+78+37)/4*2.5*10**6
likelihoods = ldcheck.pmf(rif_plates_exp_2, mu, N)
loglikelihood = np.log(likelihoods)
print(np.sum(loglikelihood))

In [ ]:
mu = .117 * 10**(-7)
N = (61+17+78+37)/4*2.5*10**6
likelihoods = ldcheck.pmf(rif_plates_exp_2, mu, N)
loglikelihood = np.log(likelihoods)
print(np.sum(loglikelihood))

In [ ]:
np.array(np.array(np.array([1],dtype='int64'))).dtype